# Image analysis

This week let's consider some basic image analysis problems.

Transformations, blurring, sharpening, etc. 

## Topic 1 - Image representation

Computer images (in particular **.png** files) are stored as $r \times c \times 3$ arrays of floating-point numbers.  $r$ represents the rows, $c$ the columns and the $3$ is for the *red*, *green* and *blue* colour fields. 

Representing a picture in this format is good for adjusting colour levels, converting to grayscale, etc.  But it is not particularly useful for resizing the image.  Or extracting features such as transitions, or edge detection.  

More afield, the subject of *computer vision* looks for techniques of automatically extracting information from images.  Think of products like *Google glass*, it is something of an attempt at a heads-up-display like in the Iron Man movie.  Its goal is to be an assistant, to identify products, people, dangers, etc. 

There are many techniques for representing real-valued functions, for example,

* Polynomials and power series 
$$f(x) \simeq a_0 + a_1 x + a_2 x^2 + \cdots $$

* Trigonometric functions and Fourier series 
$$f(x) \simeq a_0 \cos x + a_1 \sin x + a_2 \cos 2x + a_3 \sin 2x + \cdots$$

For **polynomial representation** of functions, we have two primary techniques that could be useful for us.  There is the [Stone-Weirstrauss Bernstein](https://en.wikipedia.org/wiki/Bernstein_polynomial) Theorem, which states that all functions can be approximated by polynomials.  Bernstein's approach gives an algorithm that we could implement.  Of course, the power series technique is also potentially available but it is riddled with problems:

 * Data does not always come equipped with derivatives. 
 * Raw data is rarely *analytic* i.e. the power series often will not converge to the data. 

For **trigonometric representation** of functions, there is a beautiful tool called [Fourier series](https://en.wikipedia.org/wiki/Fourier_series).  This theory provides a tool that allows one to write a (continuous) $2\pi$-periodic function

$$f(x+2\pi) = f(x)$$

as a sum of trig functions, specifically:

$$f(x) = c + a_1\sin(x) + b_1\cos(x) + a_2\sin(2x) + b_2\cos(2x) + a_3\sin(3x) + b_3\cos(3x) + \cdots$$

and the coefficients $c, a_i, b_i$ are computable via integration. 

Let's start by picking a random image from the internet. 

<img src="platypus_on_a_rock.png">

In [ ]:
%matplotlib nbagg
#%matplotlib inline
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np


## Mechanics of Fourier series

Given a continuous $2\pi$-periodic function $f : \mathbb R \to \mathbb C$ the (complex) Fourier series for $f$ is

$$f(x) = \sum_{k \in \mathbb Z} c_k e^{ikx} $$

where the coefficients are given by

$$c_k = \frac{1}{2\pi}  \int_{-\pi}^\pi f(x) e^{-ikx} dx. $$

In the case of real-valued functions we rewrite the above in terms of trig functions.  The idea is to observe that $f$ being real forces $c_{-k} = \overline{c_k}$, so one can fold the series for $f$ to

$$f(x) = c_0 + \sum_{k \in \mathbb N} c_ke^{ikx} + \overline{c_k e^{ikx}} dx$$

further writing $e^{ikx}$ and $c_k$ in terms of its real and imaginary parts gives

$$f(x) = c_0 + \sum_{k \in \mathbb N} a_k \cos(kx) + b_k \sin(kx) $$

where $c_0$ is as before $c_0 = \frac{1}{2\pi} \int_{-\pi}^\pi f(x) dx$ and

$$a_k = \frac{1}{\pi} \int_{-\pi}^\pi f(x) \cos(kx) dx$$

$$b_k = \frac{1}{\pi} \int_{-\pi}^\pi f(x) \sin(kx) dx$$

## Multi-variable Fourier series

There is an an analogous theory for multi-variable $2\pi$-periodic continuous functions in the plane.  By periodic we mean functions that satisfy

$$f(x + 2\pi, y) = f(x,y) = f(x,y+2\pi)$$

The Fourier expansion for them is a series expression

$$f(x,y) = \sum_{j,k \in \mathbb Z} c_{jk} e^{i(jx+ky)} $$

where

$$c_{jk} = \frac{1}{4\pi^2} \int_{-\pi}^\pi \int_{-\pi}^\pi f(x,y) e^{-i(jx+ky)} dx dy$$

an analogous argument shows us that

$$c_{-j,-k} = \overline{c_{jk}}$$

provided $f$ is real-valued, giving us

$$f(x,y) = c_{00} + \sum_{j,k} a_{jk} \cos(jx+ky) + b_{jk} \sin(jx+ky)$$  

with 
$$a_{jk} = \frac{1}{2\pi^2} \int_{\pi}^\pi \int_{-\pi}^{\pi} f(x,y) \cos(jx+ky) dxdy$$
$$b_{jk} = \frac{1}{2\pi^2} \int_{\pi}^\pi \int_{-\pi}^{\pi} f(x,y) \sin(jx+ky) dxdy$$

where the sum is over integer pairs $(j,k) \in \mathbb Z^2 \setminus \{0\}$, taking only one representative up to negation $\pm (j,k)$.  Perhaps the simplest way to do this would be to consider only the pairs $(j,k)$ with $j>0$, or $j=0$ with $k>0$. 

In [ ]:
## Let's code up the integrals for the a_jk and b_jk coefficients.  Let's store the coefficients
## as defaultdicts. Probably the simplest. 


### Okay, that's really slow!  Let's try something else. 

The [Fast Fourier Transform](https://en.wikipedia.org/wiki/Fast_Fourier_transform) is an alternative family of algorithms to compute Fourier series.  These algorithms are (usually) based on matrix algebra factorizations. 

Lucky for us, **numpy** has an [implementation](https://docs.scipy.org/doc/numpy/reference/routines.fft.html) implementation of the Fast Fourier Transform.  Let's try it. 

**What we need to know** the numpy 2-dimensional Fourier transform is called **fft2**.  It takes as input a numpy 2-dimensional array of floats.  

The first thing we will need to do is split our **y by x by 3** numpy array into three **y by x** arrays, one for each color.  The [**numpy.moveaxis**](https://docs.scipy.org/doc/numpy/reference/generated/numpy.moveaxis.html#numpy.moveaxis) command is useful for this step. 